# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, see [this forum post](https://carnd-forums.udacity.com/cq/viewquestion.action?spaceKey=CAR&id=29496372&questionTitle=finding-lanes---import-cv2-fails-even-though-python-in-the-terminal-window-has-no-problem-with-import-cv2) for more troubleshooting tips.**  

## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Read in an Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
"""
The following class provides interfaces to set and get 
the globals defaults for the execution of the project.
"""
class Globals(object):

    def __init__(self):
        """
        Initialize Globals. All default values are set.
        """
        #
        # Setting the golden ration as the default ration when
        # defining the region
        #
        self.ratio = 8/ 5.0;
        self.reset()
                
    def reset(self):
        """
        Resets the global defauls for the class
        """
        self.debug = False
        self.initialize_lists()
        self.set_hough_line_thresholds()
        self.set_canny_thresholds()
        self.set_slopes()
        self.set_history_limit()
        self.set_line_thickness()
        self.set_extrapolate()
        self.set_line_color()
        self._set_cvt_color()
        
    def set_debug(self):
        """
        Sets debugging on for the session
        """
        self.debug = True
        
    def is_debug(self):
        """
        Returns boolean, True is debug else False
        """
        return self.debug
        
    def initialize_lists(self):
        """
        Initialize global lists. These lists maintain to slopes,
        intercepts, max and min y values across video frames.
        """
        self.right_m_list = []
        self.left_m_list = []
        self.right_c_list = []
        self.left_c_list = []
        self.min_y_list = []
        self.max_y_list = []

    def set_line_color(self, color=[255, 0, 0]):
        """
        Set the color for the line that needs to be generated
        """
        self._color = color

    def set_hough_line_thresholds(self, rho=1, theta=np.pi/180,
                                  threshold=30, min_line_length=80, max_line_gap=110):
        """
        Define the Hough transform parameters
        """
        self._rho = rho
        self._theta = theta
        self._threshold = threshold
        self._min_line_length = min_line_length
        self._max_line_gap = max_line_gap
        
    def set_canny_thresholds(self, kernel_size=5, low_threshold=50, high_threshold=150):
        """
        Define the Canny parameters
        """
        self._kernel_size = kernel_size
        self._low_threshold = low_threshold
        self._high_threshold = high_threshold
        
    def set_slopes(self, slope_max=0.9, slope_min=0.5):
        """
        Set the slope thresholds to filter out unwanted lines
        """
        self._slope_max = slope_max
        self._slope_min = slope_min
        
    def set_history_limit(self, history=20):
        """
        Set the limit upto which historical data should be used
        to generate current averages.
        """
        self._history = history
        
    def set_line_thickness(self, thickness=10):
        """
        Set the thickness of the lines that woul dbe drawn over
        lanes.
        """
        self._thickness = thickness
        
    def set_extrapolate(self, extrapolate=True):
        """
        Set extrapolate to generate lines based on the slope and intercepts
        of the current and historical lanes. The lines will extrapolate
        over the detected lanes.
        Set to False to view hough lines.
        """
        self._extrapolate = extrapolate
        if self._extrapolate == True:
            self.set_line_thickness()
            return
        
        self.set_line_thickness(thickness=3)
        
    def _set_cvt_color(self, cvt_color = cv2.COLOR_RGB2Lab):
        """
        post trying through all the colors, found which works
        across the images and videos

        cv2.COLOR_RBG2HSV
        cv2.COLOR_RBG2HLS
        cv2.COLOR_RGB2
        cv2.COLOR_YCrCb
        cv2.COLOR_Luv
        cv2.COLOR_Lub
        """
        self._cvt_color = cvt_color
    
    def get_line_color(self):
        """ Get the current color that needs to be set """
        return self._color

    def get_hough_line_thresholds(self):
        """ 
        Get the hough transform parameters. It returns a tuple of:
            rho, theta, threshold, min_line_length, max_line_gap
        """
        return (self._rho, self._theta, self._threshold, self._min_line_length, self._max_line_gap)
    
    def get_canny_thresholds(self):
        """ 
        Get canny parameters. It returns a tuple of:
            kernel_size, low_threshold, high_threshold
        """
        return (self._kernel_size, self._low_threshold, self._high_threshold)
    
    def get_slopes(self):
        """ 
        Get slope thresholds. It returns a tuple of:
            slope_max, slope_min
        NOTE : for left lane it should be negative
        """
        return (self._slope_max, self._slope_min)
    
    def get_history_limit(self):
        """ Get the number of historical points required for averaging """
        return self._history
    
    def get_line_thickness(self):
        """ Get the thickness of the line """
        return self._thickness
    
    def should_extrapolate(self):
        """ Returns True to extrapolate lines on lanes else False """
        return self._extrapolate
    
    def get_cvt_color(self):
        return self._cvt_color
    
prj_globals = Globals()

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, prj_globals.get_cvt_color())
    # return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def extrapolate_lines(lines):
    try:
        # get all the x points for the line
        x = np.reshape(lines[:, [0, 2]], (1, len(lines) * 2))[0]
        # get all the y points for the line
        y = np.reshape(lines[:, [1, 3]], (1, len(lines) * 2))[0]
    
        # https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.lstsq.html
        # The equation for a line is y = mx + c
        # we can rewrite the equation as y = Ap, where A = [[x 1]] and p = [[m], [c]]
        # and then use lstsq to get the value of p
        A = np.vstack([x, np.ones(len(x))]).T
        m, c = np.linalg.lstsq(A, y)[0]

        x = np.array(x)
        # using the the value of m and c get all the values of y
        y = np.array(x * m + c)
    except:
        print (x, y)

    return x, y, m, c

def draw_lines(img, lines):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    # get the color for the line
    color = prj_globals.get_line_color()
    
    # get the thickness for the line
    thickness = prj_globals.get_line_thickness()
    
    # get history limits upto which the past values should be averaged
    history = prj_globals.get_history_limit()

    # get all the slopes for the line segments
    slopes = np.apply_along_axis(lambda line: (line[3] - line[1]) / (line[2] - line[0]), 2, lines)
    if prj_globals.is_debug():
        print (slopes)

    # get thresholds for the slope
    slope_max, slope_min = prj_globals.get_slopes()
    
    # the line on the right will have slope greater than 0
    right_lines = lines[np.where((slopes >= slope_min) & (slopes <= slope_max))]
    
    # the line on the left will have slope less than 0
    left_lines = lines[np.where((slopes <= -slope_min) & (slopes >= -slope_max))]
    
    if len(left_lines) == 0 or len(right_lines) == 0:
        return

    if not prj_globals.should_extrapolate():
        for line in np.array(np.concatenate((left_lines, right_lines), axis=0)):
            for x1,y1,x2,y2 in [line]:
                cv2.line(img, (x1, y1), (x2, y2), color, thickness)
        return
    
    # Extraploate lines to the the x, y, m and c values for the lines. 
    # Here m is the slope and c is the y intercept
    left_x, left_y, left_m, left_c = extrapolate_lines(left_lines)
    right_x, right_y, right_m, right_c = extrapolate_lines(right_lines)
 
    # append the current slope and intercept for left lane in globals
    prj_globals.left_m_list.append(left_m)
    prj_globals.left_c_list.append(left_c)
    
    # get the average slope and intercept for left line
    avg_left_m = np.mean(prj_globals.left_m_list[-history:])
    avg_left_c = np.mean(prj_globals.left_c_list[-history:])
    
    # append the current slope and intercept for right lane in globals
    prj_globals.right_m_list.append(right_m)
    prj_globals.right_c_list.append(right_c)
    
    # get the average slope and intercept for right line
    avg_right_m = np.mean(prj_globals.right_m_list[-history:])
    avg_right_c = np.mean(prj_globals.right_c_list[-history:])
    
    if prj_globals.is_debug():
        print(" avg_left_m = ", avg_left_m, " avg_right_m = ", avg_right_m)
        print(" avg_left_c = ", avg_left_c , " avg_right_c=", avg_right_c)
    
    # Get the minimum value for y
    min_y = np.min([np.min(left_y), np.min(right_y)])
    
    # average out the minimum y values
    prj_globals.min_y_list.append(min_y)
    avg_min_y = np.mean(prj_globals.min_y_list[-history:])
    
    # Using the minimum value of y get the top right and top left points for
    # the lines , x = (y - c)/ m
    top_right_point = np.array([((avg_min_y - avg_right_c) / avg_right_m), avg_min_y], dtype=np.int32)
    top_left_point = np.array([((avg_min_y - avg_left_c) / avg_left_m), avg_min_y], dtype=np.int32)

    # Now get the maximum value for y
    max_y = np.max([np.max(right_y), np.max(left_y)])
    
    # average out the maximum y values
    prj_globals.max_y_list.append(max_y)
    avg_max_y = np.mean(prj_globals.max_y_list[-history:])
    
    if prj_globals.is_debug():
        print("min left_y =", np.min(left_y), " min right_y =", np.min(right_y), " min y =", min_y, " avg min_y =", avg_min_y)
        print("max left_y =", np.max(left_y), " max right_y =", np.max(right_y), " max y =", max_y, " avg_max_y = ", avg_max_y)
    
    # using the maximum y value get the bottom left and bottom right points. x = (y - c)/ m
    bottom_left_point = np.array([((avg_max_y - avg_left_c) / avg_left_m), avg_max_y], dtype=np.int32)
    bottom_right_point = np.array([((avg_max_y - avg_right_c) / avg_right_m), avg_max_y], dtype=np.int32)

    # Draw the lines.
    cv2.line(img, (bottom_left_point[0], bottom_left_point[1]), (top_left_point[0], top_left_point[1]), color, thickness)
    cv2.line(img, (bottom_right_point[0], bottom_right_point[1]), (top_right_point[0], top_right_point[1]), color, thickness)
    

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the test_images directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [ ]:
import traceback

def image_processing_pipeline(input_image):
    try:
        # get the dimensions of the image
        ysize = input_image.shape[0]
        xsize = input_image.shape[1]
        
        # convert the images into grey scale
        gray = grayscale(input_image)
        if prj_globals.is_debug():
            plt.imshow(gray, cmap='gray')
            plt.show()

        # not the the image has been converted into grey scale, get 
        # edges from the images. Use canny edge algorightm. Appy gausian
        # blur before applying the canny edge detection.

        # get canny parameters
        kernel_size, low_threshold, high_threshold = prj_globals.get_canny_thresholds()
        
        # Define a kernel size and apply Gaussian smoothing
        blur_gray = gaussian_blur(gray, kernel_size)
        if prj_globals.is_debug():
            plt.imshow(blur_gray, cmap='gray')
            plt.show()

        # Define our parameters for Canny and apply
        edges = canny(blur_gray, low_threshold, high_threshold)
        if prj_globals.is_debug():
            plt.imshow(edges)
            plt.show()

        # Define a rectangle region of interest for masking the edges 
        # which are of interest
        ratio = 1 / (prj_globals.ratio)
        regions = [
                    (100, ysize),
                    ((1 - ratio) * xsize, ratio * ysize),
                    (ratio * xsize, ratio * ysize),
                    (xsize, ysize)
                   ]

        if prj_globals.is_debug():
            print (regions)

        vertices = np.array([regions], np.int32)
        masked_edges = region_of_interest(edges, vertices)
        if prj_globals.is_debug():
            plt.imshow(masked_edges)
            plt.show()
        
        # get the hogh line parameters
        rho, theta, threshold, min_line_length,max_line_gap = prj_globals.get_hough_line_thresholds()
        
        lines_image = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap)
        if prj_globals.is_debug():
            plt.imshow(lines_image)
            plt.show()

        lines_edges = weighted_img(lines_image, input_image)
        if prj_globals.is_debug():
            plt.imshow(lines_edges)
            plt.show()
    except:
        # in case of failure return the original image
        traceback.print_exc()
        return input_image

    return lines_edges
    
for input_img in os.listdir('test_images/'):
    if input_img.startswith("."):
        continue

    prj_globals.reset()
    # Read the input image
    image = mpimg.imread('test_images/%s' % input_img)
    # print the image name and statistics
    if prj_globals.is_debug():
        print('This image is:', input_img , 'type(', type(image), ') with dimesions:', image.shape)

    # show the current image
    plt.imshow(image)
    plt.show()
    
    result = image_processing_pipeline(image)
    plt.imshow(result)
    plt.show()
    #plt.savefig("final/%s" % input_img)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an `import error` when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt. Also, check out [this forum post](https://carnd-forums.udacity.com/questions/22677062/answers/22677109) for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://carnd-forums.udacity.com/display/CAR/questions/26218840/import-videofileclip-error) for more troubleshooting tips across operating systems.**

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    result = image_processing_pipeline(image)
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
prj_globals.reset()

white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
prj_globals.reset()

yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
prj_globals.reset()

challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))